# Predictive Modelling Task - 3

In this workbook, I will try to use additional signals and determine its effect on the model.

I will load the data and generate functions for forecasting and then work with stock: X3 as an example use-case.

In [6]:
import pandas as pd
import numpy as np
import warnings
# Dickey-Fuller Test
from statsmodels.tsa.stattools import adfuller
# auto corelation
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
# seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose
# ARIMA
import statsmodels.tsa as tsa
# display
from IPython.display import display
import colorama

warnings.filterwarnings('ignore')

# plot utilities
%matplotlib inline
import matplotlib.pyplot as plt
import mpld3
mpld3.enable_notebook()

def testStationarity(ts=None, displaySummary=True, displayPlot=True):
    ''' Test whether the input series is stationary or not
    '''
    # remove NAN's
    ts = ts.dropna()
    
    # create a data frame for X1
    dframe = pd.DataFrame(ts.index)
    dframe['P(t)'] = ts.values
    d = dframe['P(t)'][~dframe['P(t)'].isnull()]

    # dickyey-fuller test
    df = adfuller(ts)
    if df[0] < df[4]['1%']:
        confi = 0.99
        isStationary = True;
        strStationary = ' DFTest: Stationary'+ ' (confidence= %.2f)' % confi
        
    elif df[0] < df[4]['5%']:
        confi = 0.95
        isStationary = True;
        strStationary = ' DFTest: Stationary'+ ' (confidence= %.2f)' % confi
        
    elif df[0] < df[4]['10%']:
        confi = 0.90
        isStationary = True;
        strStationary = ' DFTest: Stationary' + ' (confidence= %.2f)' % confi
        
    else:
        confi = 0
        isStationary = False;
        strStationary = ' DFTest: Non Stationary'
        
    # moving averages
    dframe['MA_30'] = pd.Series.rolling(d, center=False, window=30).mean() # 30 -day MA
    dframe['MA_60'] = pd.Series.rolling(d, center=False, window=60).mean() # 60 -day MA
    dframe['MA_120'] = pd.Series.rolling(d, center=False, window=120).mean() # 120 -day MA

    # standard devs
    dframe['SD_30'] = pd.Series.rolling(d, center=False, window=30).std() # 30 -day SD
    dframe['SD_60'] = pd.Series.rolling(d, center=False, window=60).std() # 60 -day SD
    dframe['SD_120'] = pd.Series.rolling(d, center=False, window=120).std() # 120 -day SD
    
    if displayPlot:
        # plot
        dframe.loc[:, ['MA_30', 'MA_60', 'MA_120']].plot( title='Moving Averages - ' + ts.name + strStationary)
        dframe.loc[:, ['SD_30', 'SD_60', 'SD_120']].plot( title='Variance - ' + ts.name + strStationary)
        
    if displaySummary:
        print(df)
        print('Time series - '+ts.name + ',' + strStationary)
        
    return isStationary


def convert2Stationary(ts=None, disp=True):
    ''' Convert a non-stationary time-series to stationary using simple differencing of
        log-transformed timeseries
        The input time-series is first checked for stationarity then the conversion is done
        
        input: ts -> time-series in normal price
        output: stationary timeseries
    '''
    # log
    ts_log = np.log(ts.dropna())
    out = ts_log
    if not testStationarity(ts, displayPlot=False, displaySummary=False):
        # differencing
        out = ts_log - ts_log.shift(1)
    
    # perform test
    if disp:
        print("=== after conversion test for Stationarity===")
        testStationarity(out, displayPlot=disp, displaySummary=disp)
        
    return out



def forecast_3day_returns(ts=None, ARIMAorder=None, exogVariable=None, displaySummary=True):
    ''' Forecast 3-day returns using ARIMA modelling
    '''
    # original timeseries
    orig_tseries = ts
    # create a new time series by pushing it 3-days ahead and fill it with 0
    pushed_tseries = pd.Series(0, orig_tseries.index + pd.Timedelta('3 day'))
    # extract the last 3-days from pushed_tseries and append it to original timeseries
    tseries = orig_tseries.append(pushed_tseries[-3 : pushed_tseries.shape[0]], verify_integrity=True)
    
    # convert original timeseries X3 to stationary
    day_returns = convert2Stationary(ts, False)
    # index-0 of day_returns will be NAN due to differencing.
    # usually day_returns[original_length+1] will have -INF due to 0 insertion.

    # ARIMA order
    if ARIMAorder is None:
        ARIMAorder = (1,0,1)
    
    # Exogenous variable
    exogArrValues = None
    doexit = False
    if exogVariable is not None :
        # make sure sizes are same as original series
        assert(orig_tseries.shape[0] == exogVariable.shape[0])
        ## need to drop rows from exogVariable that correspond to rows in original-series with NAN in it
        # to do this, first create a dummy with NAN if NAN in original , 1 otherwise, 
        # then, multiple dummy to exogVariable, thereby replacing NAN if NAN in original, value otherwise
        # finally, dropna
        dummy = (ts * (ts.isnull() * 1)) + 1
        exogVariable = exogVariable.multiply(dummy, axis='rows')
        exogVariable = exogVariable.dropna()
        # make sure after dropping NAN sizes are same as day_returns
        assert(day_returns.shape[0] == exogVariable.shape[0])
        
        if 0 in exogVariable:
            # check if dummy-variable added by fillExogenousVariable has all 1's
            # dummy column name is 0, if true, then it causes an exception in ARIMA modelling, strip it off
            if sum(exogVariable[0]) == exogVariable.shape[0]:
                exogVariable = exogVariable.drop(0, axis=1)

            # check if dummy-variable dded by fillExogenousVariable has all 0's, then set exogVariable = None 
            # or issue an ERROR
            # dummy column name is 0, if true, then it causes an exception in ARIMA modelling, strip it off
            elif sum(exogVariable[0]) == 0:
                #exogVariable = exogVariable.drop(0, axis=1)
                print(colorama.Fore.RED + "WARNING: Exogeneous variable does not any values associated with any timestamps of the Input series")
                doexit = True

            # day_returns[1:] is the input for modelling, so make sure exogVariable gets the same treatment
        
        exogArrValues = exogVariable[1:]

    if doexit:
        return
    
    
    # modelling (use all data starting from index-1 day of day_returns to the lastday of the orignal timeseries)
    model = tsa.arima_model.ARIMA(day_returns.ix[1:], order=ARIMAorder, exog=exogArrValues).fit(disp=-1)    
        
    if displaySummary:
        print(model.summary())

    # index of day_returns runs from 0:day_returns.shape[0]-1, there are total of day_returns.shape[0] elements
    # we are ignoring inedx-0 while training (it might contain NAN due to differencing), hence we are only training
    # with day_returns.shape[0]-1 elements (with indexing running from 1:day_returns.shape[0]-1). 
    # Therefore the the trained-model will have a length of ((day_returns.shape[0]-1)-1)
    # For prediction, the starting index should atleast be the last fittedvalue.

    # predict
    preds = model.predict(model.fittedvalues.shape[0]-1, model.fittedvalues.shape[0]+2, exogVariable)

    ### calculate error metrics and accuracy of fittedvalues
    # Original-returns: calculate rolling_sum of log-returns over a window of 4 days, including current day
    orig_returns = day_returns[1:]
    rollsum = pd.Series.rolling(orig_returns, center=False, window=4).sum()
    orig_3day_returns =  orig_returns - rollsum.shift(-3)
    # fitted-returns: 
    rollsum = pd.Series.rolling(model.fittedvalues, center=False, window=4).sum()
    fitted_3day_returns =  model.fittedvalues - rollsum.shift(-3)
    # display dataframe

    statsFrame = pd.DataFrame(orig_returns.index)
    statsFrame['Orig Log Returns'] = orig_returns.values
    statsFrame['Pred Log Returns'] = model.fittedvalues.values
    statsFrame['AbsError'] = np.abs(orig_returns.values - model.fittedvalues.values)
    statsFrame['AbsPercentError(%)'] = np.abs((orig_returns.values - model.fittedvalues.values)/(orig_returns.values+0.00000001)) * 100 
    statsFrame['DirectionAccuracy'] = ~((orig_returns.values > 0) != (model.fittedvalues.values > 0))
    statsFrame['Orig 3-day Log returns'] = orig_3day_returns.values
    statsFrame['Pred 3-day Log returns'] = fitted_3day_returns.values
    statsFrame['3-day AbsError'] = np.abs(orig_3day_returns.values - fitted_3day_returns.values)
    statsFrame['3-day AbsPercentError(%)'] = np.abs((orig_3day_returns.values - fitted_3day_returns.values)/(orig_3day_returns.values+0.00000001)) * 100 
    statsFrame['3-day DirectionAccuracy'] = ~((orig_3day_returns.values > 0) != (fitted_3day_returns.values > 0))
    # calculate


    # create a fitted_day_returns series using index from new timeseries (created used pushed_timeseries) and fittedvalues
    fitted_day_returns = pd.Series(model.fittedvalues, index=tseries.index)
    # copy the forecast values to correct dates
    fitted_day_returns[-4 : fitted_day_returns.shape[0]] = preds
    forecast_day_returns = fitted_day_returns[-4 : fitted_day_returns.shape[0]]

    # Predicted-returns: 
    rollsum = pd.Series.rolling(forecast_day_returns, center=False, window=4).sum()
    forecast_3day_returns =  forecast_day_returns - rollsum.shift(-3)

    # Display
    futureFrame = pd.DataFrame(forecast_day_returns.index)
    futureFrame['Forecast Log Returns'] = forecast_day_returns.values
    futureFrame['Forecast 3-day Log Returns'] = forecast_3day_returns.values
    futureFrame['Long term MAE (log returns)'] = np.mean(statsFrame['AbsError'])
    futureFrame['Long term MAPE (log returns) %'] = np.mean(statsFrame['AbsPercentError(%)']/100)*100
    futureFrame['Long term DirectionAccuracy (log returns) %'] = (sum(statsFrame['DirectionAccuracy'])/statsFrame.shape[0]) * 100
    futureFrame['Long term MAE (3-day log returns)'] = np.mean(statsFrame['3-day AbsError'])
    futureFrame['Long term MAPE (3-day log returns) %'] = np.mean(statsFrame['3-day AbsPercentError(%)']/100) * 100 
    futureFrame['Long term DirectionAccuracy (3-day log returns) %'] = (sum(statsFrame['3-day DirectionAccuracy'])/statsFrame.shape[0]) * 100
    if displaySummary:
        print("=====  3-day Returns forecast for closing price of last-day in timeseries ======")
        print("           Note: The first row in the table corresponds to the last day \n           closing price of the stock")
        print("   Stock: "+ ts.name)
        display(futureFrame)
        print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')

    
    return futureFrame



# test- forecast
# convert data to time-series data while reading from csv itself
dailyPrices = pd.read_csv('dailyPrices_AtClose.csv', parse_dates='t()', index_col='t()')
#dailyPrices.index # check if the index is datetime
# re-index and set frequency='D'
dates = pd.date_range(dailyPrices.index[0], dailyPrices.index[-1], freq='D')
dailyPrices = dailyPrices.reindex(dates)
#dailyPrices.index

forecast_3day_returns(dailyPrices.X3)
    


                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 1)   Log Likelihood                6734.171
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13460.342
Time:                        20:03:44   BIC                         -13436.639
Sample:                    01-04-2005   HQIC                        -13451.781
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.772      0.440        -0.000     0.001
ar.L1.X3       0.5033      0.112      4.487      0.000         0.283     0.723
ma.L1.X3      -0.5830      0.105     -5.553      0.0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,-0.000133,-0.001179,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
1,2016-01-01,0.000486,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
2,2016-01-02,0.000374,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
3,2016-01-03,0.000318,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,-0.000133,-0.001179,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
1,2016-01-01,0.000486,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
2,2016-01-02,0.000374,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948
3,2016-01-03,0.000318,NaN,0.014031,146605.880408,52.998555,0.025966,204584.211969,35.07948



## Exogenous variables

Two files with additional signals have been provided. X1Signals_AtClose.csv and X2Signals_AtClose.csv


### X1Signals_AtClose

In [7]:
# load variable
# convert data to time-series data while reading from csv itself
x1AtClose = pd.read_csv('X1Signals_AtClose.csv', parse_dates='t()', index_col='t()')
#x1AtClose.index # check if the index is datetime
# re-index and set frequency='D'
dates = pd.date_range(dailyPrices.index[0], dailyPrices.index[-1], freq='D')
x1AtClose = x1AtClose.reindex(dates)

# forecast without x1AtClose
forecast_without = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0))

# forecast with x1AtClose
exogdf = pd.DataFrame(x1AtClose.X3, index=x1AtClose.index)
forecast_with = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0), exogVariable=exogdf)


                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6727.408
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13448.817
Time:                        20:03:51   BIC                         -13431.039
Sample:                    01-04-2005   HQIC                        -13442.396
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.684      0.494        -0.000     0.001
ar.L1.X3      -0.0598      0.019     -3.154      0.002        -0.097    -0.023
                                    Roots           

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000616,-0.001105,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
1,2016-01-01,0.000602,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
2,2016-01-02,0.000241,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
3,2016-01-03,0.000262,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6869.485
Method:                       css-mle   S.D. of innovations              0.020
Date:                Wed, 08 Jun 2016   AIC                         -13730.970
Time:                        20:03:51   BIC                         -13707.266
Sample:                    01-04-2005   HQIC                        -13722.409
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0002      0.000      0.530      0.596        -0.001     0.001
X3             0.0020      0.000     16.866      0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.006732,-0.010344,0.01338,390727.127278,64.053468,0.023916,669041.095202,62.066474
1,2016-01-01,0.001765,NaN,0.01338,390727.127278,64.053468,0.023916,669041.095202,62.066474
2,2016-01-02,0.002402,NaN,0.01338,390727.127278,64.053468,0.023916,669041.095202,62.066474
3,2016-01-03,0.006177,NaN,0.01338,390727.127278,64.053468,0.023916,669041.095202,62.066474


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


3-day forecasts made with and without additional signal X1Signal_AtClose is provided above. There is marked improvement in Direction-accuracy when using the additional signal. Also, there is a slight improvement in the log-likelihood of the data. A small reduction in MAE and MAPE is also apparent. 

I will try to capture the improvement in the following section.

In [8]:
def compareForecasts(forecast_with=None, forecast_without=None):
    ''' Compare two forecast metrics and output some statistics
        Note: forecast_with and forecast_without should be outputs of the forecast_3day_returns() function
    '''
    if forecast_with is None:
        return
    
    if forecast_without is None:
        return
    
    fwith = forecast_with.iloc[0]
    fwithout = forecast_without.iloc[0]
    sta = ((fwith.values[1:] - fwithout.values[1:])/fwithout.values[1:]) * 100

    out = pd.DataFrame(forecast_with.index)
    out['Long Term MAE (log returns) % change'] = sta[2]
    out['Long Term MAPE (log returns) % change'] = sta[3]
    out['Long Term DirectionAccuracy (log returns) % change'] = sta[4]
    out['Long Term MAE (3-day log returns) % change'] = sta[5]
    out['Long Term MAPE (3-day log returns) % change'] = sta[6]
    out['Long Term DirectionAccuracy (3-day log returns) % change'] = sta[7]
    return out[:1]

# test
out = compareForecasts(forecast_with, forecast_without)
display(out)
    

,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,-4.574243,291.269644,23.986014,-5.190278,784.470577,92.817059


The table above clearly demonstrates the improvement in using X1Signals_AtClose as additional signal to the model. Marked reduction in long term MAE's and  very good improvement in long term DirectionAccuracy. However, there is very high increase in long term MAPE.

I would like to reiterate that these statistics were calculated on prior-data not future forecasts.

I will now try to use the other additional signal, i.e. X2Signals_AtClose.csv

### X2Signals_AtClose

In [9]:
# load variable
# convert data to time-series data while reading from csv itself
x2AtClose = pd.read_csv('X2Signals_AtClose.csv', parse_dates='t()', index_col='t()')
#x1AtClose.index # check if the index is datetime
# re-index and set frequency='D'
dates = pd.date_range(dailyPrices.index[0], dailyPrices.index[-1], freq='D')
x2AtClose = x2AtClose.reindex(dates)

# forecast without x2AtClose
forecast_without = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0))

# forecast with x12tClose
exogdf = pd.DataFrame(x2AtClose.X3, index=x2AtClose.X3)
forecast_with = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0), exogVariable=exogdf)

                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6727.408
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13448.817
Time:                        20:04:28   BIC                         -13431.039
Sample:                    01-04-2005   HQIC                        -13442.396
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.684      0.494        -0.000     0.001
ar.L1.X3      -0.0598      0.019     -3.154      0.002        -0.097    -0.023
                                    Roots           

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000616,-0.001105,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
1,2016-01-01,0.000602,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
2,2016-01-02,0.000241,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
3,2016-01-03,0.000262,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


TypeError: type object argument after * must be a sequence, not map

In [10]:
x2AtClose.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,3.625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,3.500,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


x2AtClose has many NAN's, this is posing a problem at the moment. One of the easiest ways of tackling NAN is via dropping them. However, these additional signals are exogenous-variables and dropping timestamps based on exogenous-variables is not correct.  So, I need to figure out a way to remove these NANs or nullify them. 

I can fill NAN's within these additional signals using moving-average, backfill, forwardfill, etc. However, as seen in the current case, the additional signal has just one valid data point with rest all being NANs. Averaging, backfilling, forwardfilling does not seem appropriate!

I will try and use a dummy variable as recommended in [1].

References;
* 1: https://github.com/statsmodels/statsmodels/issues/2551


In [11]:
# exogenous variable
exvar = x2AtClose.X3
# create a dummy variable
dummy = pd.Series(0, index=exvar.index)
# replace with 0's and 1's wrt NAN from exvar
dummy = ~exvar.isnull() * 1
# replace NAN with 0 in exvar
exvar = exvar.fillna(0)
# concat
exvar_new = pd.concat([exvar, dummy], axis=1)
exvar_new

,X3,X3
2005-01-03,0.000,0
2005-01-04,0.000,0
2005-01-05,0.000,0
2005-01-06,0.000,0
2005-01-07,0.000,0
2005-01-08,0.000,0
2005-01-09,0.000,0
2005-01-10,0.000,0
2005-01-11,0.000,0
2005-01-12,0.000,0


In [12]:
# forecast with x2AtClose
forecast_with = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0), exogVariable=exvar_new)

                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6730.480
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13450.959
Time:                        20:04:48   BIC                         -13421.330
Sample:                    01-04-2005   HQIC                        -13440.258
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0004      0.000      1.093      0.275        -0.000     0.001
X3             0.0068      0.004      1.751      0.080        -0.001     0.014
X3            -0.0284      0.015     -1.954      0.0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000803,-0.001626,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
1,2016-01-01,0.000788,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
2,2016-01-02,0.000407,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
3,2016-01-03,0.000431,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


By creating a dummy variable, I was able to takecare of NAN's in the exogenous variable. I will wrap this into a function.

In [13]:
def fillExogenousVariable(exvar=None):
    ''' Check if the exogenous variable series has NANs, concatenate dummy variable if required 
        if yes, then concat a dummy variable to help handle these NANs
    '''
    # dummy variable
    dummy = pd.Series(True, index=exvar.index)
    for col in exvar:
        dummy = np.logical_and(dummy, ~exvar[col].isnull())
        
    
    # replace NANs in exvar with 0
    ex = exvar
    ex[~dummy] = 0
    # convert bool to int
    dummy = dummy * 1
    # output
    out = pd.concat([ex, dummy], axis=1)
    return out

    
# test
tmp1 = x2AtClose.X3
tmp2 = x2AtClose.X4
ex = pd.concat([tmp1, tmp2], axis=1)

ex1 = fillExogenousVariable(ex)
print(ex)
print(ex1)

             X3   X4
2005-01-03  0.0  0.0
2005-01-04  0.0  0.0
2005-01-05  0.0  0.0
2005-01-06  0.0  0.0
2005-01-07  0.0  0.0
2005-01-08  0.0  0.0
2005-01-09  0.0  0.0
2005-01-10  0.0  0.0
2005-01-11  0.0  0.0
2005-01-12  0.0  0.0
2005-01-13  0.0  0.0
2005-01-14  0.0  0.0
2005-01-15  0.0  0.0
2005-01-16  0.0  0.0
2005-01-17  0.0  0.0
2005-01-18  0.0  0.0
2005-01-19  0.0  0.0
2005-01-20  0.0  0.0
2005-01-21  0.0  0.0
2005-01-22  0.0  0.0
2005-01-23  0.0  0.0
2005-01-24  0.0  0.0
2005-01-25  0.0  0.0
2005-01-26  0.0  0.0
2005-01-27  0.0  0.0
2005-01-28  0.0  0.0
2005-01-29  0.0  0.0
2005-01-30  0.0  0.0
2005-01-31  0.0  0.0
2005-02-01  0.0  0.0
...         ...  ...
2015-12-02  0.0  0.0
2015-12-03  0.0  0.0
2015-12-04  0.0  0.0
2015-12-05  0.0  0.0
2015-12-06  0.0  0.0
2015-12-07  0.0  0.0
2015-12-08  0.0  0.0
2015-12-09  0.0  0.0
2015-12-10  0.0  0.0
2015-12-11  0.0  0.0
2015-12-12  0.0  0.0
2015-12-13  0.0  0.0
2015-12-14  0.0  0.0
2015-12-15  0.0  0.0
2015-12-16  0.0  0.0
2015-12-17  0

Now, back to experiements. Using X2Signal_AtClose as additional signal, we get:

In [14]:
# forecast without x2AtClose
forecast_without = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0))

## forecast with x2AtClose
# convert exogenous series to dataframe
exvar = pd.DataFrame(x2AtClose.X3, index=x2AtClose.index)
exvar = fillExogenousVariable(exvar)
forecast_with = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0), exogVariable=exvar)

compareForecasts(forecast_with, forecast_without)

                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6727.408
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13448.817
Time:                        20:05:03   BIC                         -13431.039
Sample:                    01-04-2005   HQIC                        -13442.396
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.684      0.494        -0.000     0.001
ar.L1.X3      -0.0598      0.019     -3.154      0.002        -0.097    -0.023
                                    Roots           

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000616,-0.001105,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
1,2016-01-01,0.000602,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
2,2016-01-02,0.000241,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
3,2016-01-03,0.000262,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6730.480
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13450.959
Time:                        20:05:03   BIC                         -13421.330
Sample:                    01-04-2005   HQIC                        -13440.258
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0004      0.000      1.093      0.275        -0.000     0.001
X3             0.0068      0.004      1.751      0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000803,-0.001626,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
1,2016-01-01,0.000788,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
2,2016-01-02,0.000407,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757
3,2016-01-03,0.000431,NaN,0.014014,124344.62042,52.312139,0.025215,105176.54296,39.703757


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,-0.049906,24.51727,1.258741,-0.03841,39.043115,23.344557


The above table shows the change in metrics when using X2Signal_AtClose as additional signal compared to not using it. There is not much improvement at all compared to the improvement gained by using X1Signal_AtClose as additional signal.


### Combined X1Signal_AtClose and X2Signal_AtClose

In [15]:
# forecast without additional signal
forecast_without = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0))

## forecast with x1AtClose & x2AtClose
# convert exogenous series to dataframe
exvar = pd.DataFrame(x1AtClose.X3, index=x1AtClose.index)
exvar['X31'] = x2AtClose.X3
exvar = fillExogenousVariable(exvar)
forecast_with = forecast_3day_returns(dailyPrices.X3, ARIMAorder=(1,0,0), exogVariable=exvar)

compareForecasts(forecast_with, forecast_without)

                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6727.408
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13448.817
Time:                        20:05:18   BIC                         -13431.039
Sample:                    01-04-2005   HQIC                        -13442.396
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0003      0.000      0.684      0.494        -0.000     0.001
ar.L1.X3      -0.0598      0.019     -3.154      0.002        -0.097    -0.023
                                    Roots           

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000616,-0.001105,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
1,2016-01-01,0.000602,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
2,2016-01-02,0.000241,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306
3,2016-01-03,0.000262,NaN,0.014021,99861.344553,51.66185,0.025225,75643.114964,32.189306


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
                              ARMA Model Results                              
Dep. Variable:                     X3   No. Observations:                 2768
Model:                     ARMA(1, 0)   Log Likelihood                6768.462
Method:                       css-mle   S.D. of innovations              0.021
Date:                Wed, 08 Jun 2016   AIC                         -13524.923
Time:                        20:05:19   BIC                         -13489.368
Sample:                    01-04-2005   HQIC                        -13512.082
                         - 12-31-2015                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.0004      0.000      1.120      0.263        -0.000     0.001
X3             0.0044      0.001      8.796      0

,0,Forecast Log Returns,Forecast 3-day Log Returns,Long term MAE (log returns),Long term MAPE (log returns) %,Long term DirectionAccuracy (log returns) %,Long term MAE (3-day log returns),Long term MAPE (3-day log returns) %,Long term DirectionAccuracy (3-day log returns) %
0,2015-12-31,0.000912,-0.001714,0.013909,191599.047753,52.67341,0.024824,127909.704258,41.148844
1,2016-01-01,0.000893,NaN,0.013909,191599.047753,52.67341,0.024824,127909.704258,41.148844
2,2016-01-02,0.000390,NaN,0.013909,191599.047753,52.67341,0.024824,127909.704258,41.148844
3,2016-01-03,0.000430,NaN,0.013909,191599.047753,52.67341,0.024824,127909.704258,41.148844


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,-0.800947,91.865079,1.958042,-1.587824,69.096294,27.833895


Using combined X1Signal_AtClose and X2Signal_AtClose as additional signal did not improve the metrics of our model by that much, when compared to the improvement gained by using just X1Signal_AtClose as additional signal. I suspect this is due to the fact that X2Signal_AtClose has too many NANs and the way I handle NANs in exogenous variables. When I attempt to nullify the NAN in X2Signal_AtClose, I also nullify the effect of X1Signal_AtClose at those timestamps. It appears that for timestamps in X2Signal_AtClose that have NAN, the corresponding timestamp in X1Signal_AtClose has lot more useful information, that is being lost at the moment. Therefore, for this dataset, I think X1Signal_AtClose alone as additional signal will improve the metrics far more than using other additional signals.

One takeaway from the above experiement is that the external variable X1Signal_AtClose appears to have more positive impact on stock:X3 compared to X2Signal_AtClose. Looking closely at X2Signal_AtClose, the data-points are very sparse which alludes to the fact that these events are very rare, hence it might have little correlation or no-effect on the returns of stock:X3.


### Perform additional signal experiement on all stocks

I will now perform addition signal experiement on all stocks and display just the change-in-metrics table for each stock. 


In [16]:

# process
for stock in dailyPrices:
    print('*******************************    Stock :  ' + dailyPrices[stock].name + '   ****************************')
    # forecast without additional signal
    forecast_without = forecast_3day_returns(dailyPrices[stock], ARIMAorder=(1,0,0), displaySummary=False)
    
    # forecast with x1AtClose
    # convert exogenous series to dataframe
    exvar =[]
    exvar = pd.DataFrame(x1AtClose[stock], index=x1AtClose.index)
    exvar = fillExogenousVariable(exvar)
    forecast_with1 = forecast_3day_returns(dailyPrices[stock], ARIMAorder=(1,0,0), exogVariable=exvar, displaySummary=False)
    
    # forecast with x2AtClose
    # convert exogenous series to dataframe
    exvar =[]
    exvar = pd.DataFrame(x2AtClose[stock], index=x2AtClose.index)
    exvar = fillExogenousVariable(exvar)
    forecast_with2 = forecast_3day_returns(dailyPrices[stock], ARIMAorder=(1,0,0), exogVariable=exvar, displaySummary=False)
    
    ## forecast with x1AtClose & x2AtClose
    # convert exogenous series to dataframe
    exvar =[]
    exvar = pd.DataFrame(x1AtClose[stock], index=x1AtClose.index)
    exvar['X31'] = x2AtClose.X3
    exvar = fillExogenousVariable(exvar)
    forecast_with12 = forecast_3day_returns(dailyPrices[stock], ARIMAorder=(1,0,0), exogVariable=exvar, displaySummary=False)
    
    print('=============   X1Signal_AtClose as additional: Change in metrics with & without')
    out = []
    out = compareForecasts(forecast_with1, forecast_without)
    display(out)
    print('=============   X2Signal_AtClose as additional: Change in metrics with & without')
    out =[]
    out = compareForecasts(forecast_with2, forecast_without)
    display(out)
    print('=============   X1Signal_AtClose + X2Signal_AtClose as additional: Change in metrics with & without')
    out =[]
    out = compareForecasts(forecast_with12, forecast_without)
    display(out)
    print('*******************************    Stock :  ' + dailyPrices[stock].name + '   ****************************')

*******************************    Stock :  X1   ****************************
=============   X1Signal_AtClose as additional: Change in metrics with & without


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,-1.825648,45.860303,12.115385,-3.123716,183.542838,48.189415


=============   X2Signal_AtClose as additional: Change in metrics with & without


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,0.067774,15.463877,-0.192308,0.266809,-49.395109,-6.406685


=============   X1Signal_AtClose + X2Signal_AtClose as additional: Change in metrics with & without


...

=============   X2Signal_AtClose as additional: Change in metrics with & without


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,0.098149,-27.964517,-0.417537,1.014049,-27.850203,0.603622


=============   X1Signal_AtClose + X2Signal_AtClose as additional: Change in metrics with & without


,0,Long Term MAE (log returns) % change,Long Term MAPE (log returns) % change,Long Term DirectionAccuracy (log returns) % change,Long Term MAE (3-day log returns) % change,Long Term MAPE (3-day log returns) % change,Long Term DirectionAccuracy (3-day log returns) % change
0,0,-0.419941,11.083716,2.713987,-0.863638,2.538797,3.420523


*******************************    Stock :  X56   ****************************
